In [1]:
import pandas as pd
import random
import numpy as np

In [2]:
user_items_final = pd.read_parquet("../Datasets_final/user_items_final.parquet.gz")
steam_games_final = pd.read_parquet("../Datasets_final/steam_games_final.parquet.gz")
user_reviews_final = pd.read_parquet("../Datasets_final/user_reviews_final.parquet.gz")

-------------------------

# Función developer

In [3]:
def developer(desarrollador: str):
    """Esta función devuelve un diccionario con la cantidad de items por año y el porcentaje de juegos free que hay en cada año. Recibe como parámetro el desarrollador en formato str."""
    unique_developers = steam_games_final['developer'].dropna().unique()

    if desarrollador in unique_developers:
        # Filtramos el DF por developer seleccionado y nos quedamos con las columnas 'releace_year' y 'price', borrando los nulos que haya en 'price':
        df_selected = steam_games_final.loc[steam_games_final["developer"] == desarrollador, ["release_year", "price"]].dropna(subset=["price"])

        # Calculamos la cantidad total de items y free items por año
        df_summary = df_selected.assign(free_items=(df_selected['price'] == 0.00).astype(int)).groupby('release_year').agg(quantity_total=('price', 'count'), free_items=('free_items', 'sum')).reset_index()

        # Calculamos el porcentaje de juegos free por año
        df_summary['free_per_year'] = (df_summary['free_items'] * 100 / df_summary['quantity_total']).round(2).astype(str) + '%'

        # Ordenamos por año y seleccionar las columnas necesarias para el resultado final
        resultado_df = df_summary.sort_values(by='release_year').reset_index(drop=True)[["release_year", "quantity_total", "free_per_year"]].rename(columns={"release_year": "Año", "quantity_total": "Cantidad de items", "free_per_year": "Contenido free"})

        # Finalmente convertimos el DF a un diccionario
        resultado_dict = resultado_df.to_dict(orient='records')

        return resultado_dict
    else:
        return f"El valor del parámetro debe ser str y el desarrollador debe ser alguno de los desarrolladores disponibles en Steam. Las desarrolladoras disponibles son: {', '.join(unique_developers)}"

Probamos developer:

In [4]:
developer("Treyarch")

[{'Año': '2008', 'Cantidad de items': 1, 'Contenido free': '0.0%'},
 {'Año': '2010', 'Cantidad de items': 1, 'Contenido free': '0.0%'},
 {'Año': '2011', 'Cantidad de items': 4, 'Contenido free': '0.0%'},
 {'Año': '2012', 'Cantidad de items': 1, 'Contenido free': '0.0%'},
 {'Año': '2013', 'Cantidad de items': 28, 'Contenido free': '0.0%'},
 {'Año': '2014', 'Cantidad de items': 8, 'Contenido free': '0.0%'},
 {'Año': '2015', 'Cantidad de items': 6, 'Contenido free': '0.0%'},
 {'Año': '2016', 'Cantidad de items': 4, 'Contenido free': '0.0%'},
 {'Año': '2017', 'Cantidad de items': 4, 'Contenido free': '0.0%'},
 {'Año': 'Unknown', 'Cantidad de items': 1, 'Contenido free': '0.0%'}]

----------------------------------------------

# Función UserForGenre

In [5]:
def UserForGenre(genero: str):
    # Verificamos si el género está presente en los datos disponibles
    if genero not in steam_games_final['genres'].unique():
        return "El género ingresado no está disponible en Steam."
    
    # Realizamos el merge de los dataframes correspondiente
    df_merge = pd.merge(user_items_final, steam_games_final, on="item_id", how="inner")
    
    # Filtramos los datos por el género y los años disponibles
    df_filtered = df_merge[(df_merge['genres'] == genero) & (df_merge['release_year'] != 'Unknown')]
    
    # Verificamos si no hay datos para el género en los años disponibles, en tal caso retornamos un mensaje
    if df_filtered.empty:
        return f"No hay datos para el género {genero} en los años disponibles."

    # Calculamos las horas jugadas en vase a los segundos de la columna 'playtime_forever' creando una nueva columna
    df_filtered["playtime_hours"] = (df_filtered["playtime_forever"] / 3600).round(2)
    
    # Agrupamos por usuario y calculamos la suma de horas jugadas de la columna nueva 'playtime_hours'
    df_grouped = df_filtered.groupby("user_id")['playtime_hours'].sum().reset_index()
    
    # Encuentramos al usuario con más horas jugadas
    max_hours_user = df_grouped.loc[df_grouped['playtime_hours'].idxmax(), 'user_id']
    
    # Agrupamos por año y calculamos la suma de horas jugadas por año
    df_grouped_years = df_filtered.groupby("release_year")["playtime_hours"].sum().reset_index()
    
    # Conviertimos los datos agrupados por año en un diccionario
    list_of_dictionary = df_grouped_years.to_dict(orient="records")

    # Retornamos finalmente un diccionario con el usuario con más horas jugadas y las horas jugadas por año
    return {f"Usuario con más horas jugadas para Género {genero}": max_hours_user, "Horas jugadas por año": list_of_dictionary}



Probamos UserForGenre:

In [6]:
UserForGenre('Casual')

C:\Users\patri\AppData\Local\Temp\ipykernel_13792\3838551150.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["playtime_hours"] = (df_filtered["playtime_forever"] / 3600).round(2)


{'Usuario con más horas jugadas para Género Casual': 'REBAS_AS_F-T',
 'Horas jugadas por año': [{'release_year': '1983', 'playtime_hours': 0.93},
  {'release_year': '1984', 'playtime_hours': 0.09},
  {'release_year': '1991', 'playtime_hours': 0.38},
  {'release_year': '1992', 'playtime_hours': 0.6699999999999999},
  {'release_year': '1993', 'playtime_hours': 6.09},
  {'release_year': '1994', 'playtime_hours': 12.72},
  {'release_year': '1995', 'playtime_hours': 0.98},
  {'release_year': '1996', 'playtime_hours': 8.46},
  {'release_year': '1997', 'playtime_hours': 8.95},
  {'release_year': '1998', 'playtime_hours': 3.52},
  {'release_year': '1999', 'playtime_hours': 174.97},
  {'release_year': '2000', 'playtime_hours': 14.72},
  {'release_year': '2001', 'playtime_hours': 1.71},
  {'release_year': '2002', 'playtime_hours': 16.79},
  {'release_year': '2003', 'playtime_hours': 0.81},
  {'release_year': '2004', 'playtime_hours': 6.66},
  {'release_year': '2005', 'playtime_hours': 11.2000000

-----------------------------

# Función sentiment_analysis

In [7]:
def sentiment_analysis(empresa_desarrolladora: str):
    """Devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento positivo o negativo como valor. Recibe como parametro la empresa desarrolladora de tipo str"""
    unique_developers = steam_games_final['developer'].dropna().unique()

        # Verificamos si el parametro es string y si se encuentra dentro de los 'developer' existentes 
    if isinstance(empresa_desarrolladora, str) and empresa_desarrolladora in unique_developers:
        # Filtrarmos el DF user_reviews_final por el 'item_id' de los juegos de la empresa desarrolladora(developer)
        df_filtered = user_reviews_final[user_reviews_final['item_id'].isin(steam_games_final.loc[steam_games_final['developer'] == empresa_desarrolladora, 'item_id'])]

        # Contamos la cantidad de registros por análisis de sentimiento convirtiendolo en diccionario
        sentiment_counts = df_filtered['sentiment_analysis'].value_counts().to_dict()

        # Obtenemos la cantidad de registros positivos y negativos
        positive_count = sentiment_counts.get(2)
        negative_count = sentiment_counts.get(0)

        result = {empresa_desarrolladora: [f"Negative: {negative_count}", f"Positive: {positive_count}"]}
        return result
    else:
        return f"El valor del parámetro debe ser str y el desarrollador debe ser alguno de los desarrolladores disponibles en Steam. Los desarrolladores disponibles son: {', '.join(unique_developers)}"


Probamos sentiment_analysis:

In [8]:
# Probamos
sentiment_analysis("Tamationgames")

{'Tamationgames': ['Negative: 1', 'Positive: None']}

-------------------------

## Función game_recommendation:

In [9]:


def game_recommendation(id_producto: int):
    """Esta función recibe como parámetro la ID de un juego y devuelve 5 juegos aleatorios recomendables si encuentra la ID en el sistema, y 5 juegos aleatorios si no la encuentra. Recibe un valor entero."""

        # Verificamos si el parametro es de tipo entero(int)
    if isinstance(id_producto, int):
        # Realizamos las filtraciones y creamos una nueva columna 'is_recommend'
        df_merge = pd.merge(user_reviews_final, steam_games_final, on="item_id", how="inner")
        df_merge["is_recommend"] = ((df_merge["recommend"]) & (df_merge["sentiment_analysis"] == 2))
        
        df_recommend = df_merge[(df_merge["item_id"] != id_producto) & (df_merge['genres'].isin(df_merge['genres'][df_merge['item_id'] == id_producto])) & (df_merge["is_recommend"] == 1)]
        
        # Verificamos que no este vacio el DF filtrado y en ese caso devolvemos recomendaciones aleatorias
        if not df_recommend.empty:
            sampled_recommend = random.sample(df_recommend["item_name"].tolist(), min(5, len(df_recommend)))
            return {"Los juegos recomendados son": sampled_recommend}
        
        else:
            # Por el contrario obtenemos juegos aleatorios si no hay recomendaciones
            games = df_merge["item_name"].unique()
            sampled_games = random.sample(games.tolist(), min(5, len(games)))
            return {"El item_id del juego no se encuentra en la base de datos o no hay juegos recomendados. Se mostrarán juegos aleatorios": sampled_games}    

    else:
        return "El ID debe ser un tipo de dato entero."


Probamos game_recommendation:

In [10]:
game_recommendation(10)

{'Los juegos recomendados son': ['Warframe',
  'EDGE',
  'Call of Duty®: Black Ops II',
  'Realm of the Mad God',
  'Team Fortress 2']}

--------------------------------------


## Función best_developer_year

In [11]:
def best_developer_year(año: str):
    """Esta funcion devuelve el top 3 de desarrolladoras con juegos mas recomendados por usuarios para el año dado. Recibe como parametro el año en string"""
    
    # Verificamos si el parametros años no esta presente en la columna 'release_year'
    if año not in steam_games_final['release_year'].unique():
        return f"El año {año} no está disponible en Steam o no es válido."

    # Realizamos el merge de los df correspondientes y filtramos
    df_merge = pd.merge(user_reviews_final, steam_games_final, on="item_id", how="inner")
    df_merge = df_merge.drop_duplicates(subset=['user_id', 'item_id'], keep='first')
    df_filtered = df_merge[(df_merge["release_year"] == año) & (df_merge["recommend"] == True) & (df_merge["sentiment_analysis"] == 2)]

    # Verificamos que no esta vacio y en caso de estarlo retornamos mensaje
    if df_filtered.empty:
        return f"No hay datos para el año {año} con las condiciones especificadas."

    # Agrupamos por desarrollador (developer) y calculamos la cantidad de registros
    df_size_rows = df_filtered.groupby('developer').size().nlargest(3).reset_index(name='number_rows')
    result = [{"Puesto " + str(i + 1): row["developer"]} for i, row in df_size_rows.iterrows()]
    return result


Probamos best_developer_year:

In [12]:
best_developer_year('2017')

[{'Puesto 1': 'Smartly Dressed Games'},
 {'Puesto 2': 'Freejam'},
 {'Puesto 3': 'Studio Wildcard,Instinct Games,Efecto Studios,Virtual Basement LLC'}]